In [ ]:
from ultralytics import YOLO
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches

# Load the YOLOv8 model
def load_model(model_path):
    return YOLO(model_path)

# Perform inference on a single image
def predict_single_image(model, image_path, conf_threshold=0.5):
    # Run inference
    results = model(image_path, conf=conf_threshold)

    # Extract results
    detections = results[0]
    boxes = detections.boxes.xyxy.cpu().numpy()  # Bounding box coordinates (xmin, ymin, xmax, ymax)
    scores = detections.boxes.conf.cpu().numpy()  # Confidence scores
    labels = detections.boxes.cls.cpu().numpy().astype(int)  # Class labels

    return boxes, labels, scores

# Visualize predictions
def visualize_predictions(image_path, boxes, labels, scores, label_map):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # Convert from BGR to RGB for Matplotlib

    fig, ax = plt.subplots(1, figsize=(12, 8))
    ax.imshow(image)

    for box, label, score in zip(boxes, labels, scores):
        xmin, ymin, xmax, ymax = box
        rect = patches.Rectangle(
            (xmin, ymin), xmax - xmin, ymax - ymin,
            linewidth=2, edgecolor='r', facecolor='none'
        )
        ax.add_patch(rect)
        ax.text(
            xmin, ymin - 10,
            f"{label_map[label]}: {score:.2f}",
            color='red', fontsize=12, bbox=dict(facecolor='yellow', alpha=0.5)
        )

    plt.axis('off')
    plt.show()

# Main function to test YOLOv8n model on a single image
def test_yolov8n_on_image(model_path, image_path, label_map, conf_threshold=0.5):
    # Load the YOLOv8 model
    model = load_model(model_path)

    # Perform prediction
    boxes, labels, scores = predict_single_image(model, image_path, conf_threshold)

    # Visualize predictions
    visualize_predictions(image_path, boxes, labels, scores, label_map)

# Label map (e.g., user-defined mapping from label IDs to class names)
label_map = {
    0: "Hyphaene petersiana",
    1: "Philenoptera violacea",
    2: "Combretum imberbe"
}

# Paths
model_path = "datasets/all/results/yolov8n_model.pt"  # Replace with your YOLOv8 model path
image_path = "datasets/all/val/images/CC_00394.jpg"  # Replace with your test image path

# Run the test
test_yolov8n_on_image(model_path, image_path, label_map, conf_threshold=0.5)
